For this exercise I pull some data from https://data.nasdaq.com/ (formerly Quandl API).

I registered for a free account on the https://data.nasdaq.com/ website.

Afterwards, I received a unique API key, that I stored in a `.env` file. The library `python-dotenv` is required to access it in this notebook. The assignment indicated to never submit keys to source control so I make sure to have a `.gitignore` file with a line for `.env` added to it. 

In [5]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

T7zoFaLjGTvxqunQR2Ei


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [6]:
# First, import the relevant modules
import requests
import pandas as pd

I used the nasdaq api at `https://data.nasdaq.com/api/v3/`. The relevant paremeters for the API are here https://docs.data.nasdaq.com/docs/parameters-2

We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

Below I modified the link to the API to retrieve the time period and the stock required for this exercise.

In [11]:
# Now, call the Nasdaq API and pull out a small sampe of the data (only one day) to get a glimpse
# I return data for 2017 as specified in exercise 1 of the assignment
# into the JSON structure that will be returned
TICKER = "AFX_X"
start_date = '2017-01-01'
end_date = '2017-12-31'
URL_nasdaq = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=" + start_date + "&end_date=" + end_date + "&api_key=" + API_KEY
data = requests.get(URL_nasdaq)


The JSON file is structured as a Python dictionary. The file has more information than we need, so I take the section I need and turn it into a data frame using Pandas.

In [27]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
data_dict = data.json()
#print(data_dict)

# I turn data structure (dictionary) into a data frame
col_names_nasdaq = data_dict['dataset']['column_names']
dict_values_nasdaq = data_dict['dataset']['data']
nasdaq_df = pd.DataFrame(dict_values_nasdaq, columns = col_names_nasdaq)

print(nasdaq_df.head())
print(col_names_nasdaq)

         Date   Open   High    Low  Close  Change  Traded Volume   Turnover  \
0  2017-12-29  51.76  51.94  51.45  51.76     NaN        34640.0  1792304.0   
1  2017-12-28  51.65  51.82  51.43  51.60     NaN        40660.0  2099024.0   
2  2017-12-27  51.45  51.89  50.76  51.82     NaN        57452.0  2957018.0   
3  2017-12-22  51.05  51.50  50.92  51.32     NaN        71165.0  3641949.0   
4  2017-12-21  51.16  51.52  50.90  51.40     NaN       120649.0  6179433.0   

  Last Price of the Day Daily Traded Units Daily Turnover  
0                  None               None           None  
1                  None               None           None  
2                  None               None           None  
3                  None               None           None  
4                  None               None           None  
['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [28]:
# Exercise 3 - Calculate the highest and lowest opening prices for the stock in this period
print('Highest opening stock price: ' + str(nasdaq_df['Open'].max()))
print('Lowest opening stock price: ' + str(nasdaq_df['Open'].min()))


Highest opening stock price: 53.11
Lowest opening stock price: 34.0


In [29]:
# Exercise 4 - Calculate largest change in any one day based on High & Low price
nasdaq_df['Diff_daily'] = nasdaq_df['High'] - nasdaq_df['Low']
print('Largest increase in price stock in a day: ' + str(round(nasdaq_df['Diff_daily'].max(), 4)))
print('Largest decrease in price stock in a day: ' + str(round(nasdaq_df['Diff_daily'].min(), 4)))
nasdaq_df['Change_daily'] = abs(nasdaq_df['High'] - nasdaq_df['Low'])/nasdaq_df['Low']*100
print('Largest % change in price stock in a day: ' + str(round(nasdaq_df['Change_daily'].max())) + ' %')


Largest increase in price stock in a day: 2.81
Largest decrease in price stock in a day: 0.19
Largest % change in price stock in a day: 6 %


In [30]:
# Exercise 5 - Calculate largest change in stock price between any two days (based on Closing price)
diff_between_days = []
change_between_days = []

for i in range(0, len(nasdaq_df.index)-1):
    diff = nasdaq_df['Open'][i] - nasdaq_df['Open'][i+1]
    change = abs(nasdaq_df['Open'][i+1] - nasdaq_df['Open'][i])/nasdaq_df['Open'][i]*100
    diff_between_days.append(diff)
    change_between_days.append(change)

print('Largest difference in stock price between two days: ' + str(round(max(diff_between_days),4)))
print('Largest % change in stock price between two days: ' + str(round(max(change_between_days),4)) + ' %')


Largest difference in stock price between two days: 1.78
Largest % change in stock price between two days: 9.281 %


In [31]:
# Exercise 6 - Calculate average trading volume
avg_vol = sum(nasdaq_df['Traded Volume'])/len(nasdaq_df.index)

print('Average trading volume: ' + str(round(avg_vol,4)))
print('Average trading volume (calculate by pandas): ' + str(round(nasdaq_df['Traded Volume'].mean(), 4)))

Average trading volume: 89124.3373
Average trading volume (calculate by pandas): 89124.3373


In [32]:
# Exercise 6 - Calculate median trading volume
def median_original(x):
    x_sorted = sorted(x)
    index_len = len(x.index)
    if index_len % 2 == 0:
        index_1 = index_len//2
        index_2 = index_len//2 + 1
        y = (x_sorted[index_1] + x_sorted[index_2])/2
    else:
        index_3 = index_len//2
        y = x_sorted[index_3]
    return(y)

print('Median trading value with original function: ' + str(median_original(nasdaq_df['Traded Volume'])))
print('Median trading value with pandas: ' + str(nasdaq_df['Traded Volume'].median()))


Median trading value with original function: 76286.0
Median trading value with pandas: 76286.0
